In [1]:
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

In [2]:
higgs_whole = pd.read_csv('./Predictions & Whole Data/HIGGS.csv', header=None, skiprows = range(0,10500000))

In [3]:
higgs_whole.shape

(500000, 29)

In [4]:
higgs_whole = higgs_whole[0].astype(int)
higgs_whole

0         0
1         1
2         1
3         0
4         0
         ..
499995    1
499996    1
499997    1
499998    0
499999    0
Name: 0, Length: 500000, dtype: int32

In [5]:
test_set = higgs_whole
cb_preds = pd.read_csv('./Predictions & Whole Data/optimal_catboost_preds.csv', header=None, dtype=int)
xgb_preds = pd.read_csv('./Predictions & Whole Data/optimized_xgboost_preds_on_big_test_set.csv', header=None, dtype=int)

In [6]:
mc_data = pd.DataFrame()
mc_data['ground_truth'] = higgs_whole
mc_data['y_pred_cb'] = cb_preds.values
mc_data['y_pred_xgb'] = xgb_preds.values
mc_data.head()

,ground_truth,y_pred_cb,y_pred_xgb
0,0,0,0
1,1,1,1
2,1,1,1
3,0,0,0
4,0,1,0


In [7]:
mc_data.loc[mc_data['ground_truth']==mc_data['y_pred_cb'],'correct_cb']="Yes"
mc_data.loc[mc_data['ground_truth']==mc_data['y_pred_xgb'],'correct_xgb']="Yes"
mc_data.loc[mc_data['ground_truth']!=mc_data['y_pred_cb'],'correct_cb']="No"
mc_data.loc[mc_data['ground_truth']!=mc_data['y_pred_xgb'],'correct_xgb']="No"
mc_data.head()

,ground_truth,y_pred_cb,y_pred_xgb,correct_cb,correct_xgb
0,0,0,0,Yes,Yes
1,1,1,1,Yes,Yes
2,1,1,1,Yes,Yes
3,0,0,0,Yes,Yes
4,0,1,0,No,Yes


In [8]:
nr_corr_cb_corr_xgb=0
nr_corr_cb_incorr_xgb=0
nr_incorr_cb_corr_xgb=0
nr_incorr_cb_incorr_xgb=0

for index, row in mc_data.iterrows():
    
    if  row['correct_cb']== "Yes" and  row['correct_xgb']=="Yes":
        nr_corr_cb_corr_xgb += 1
        
    elif row['correct_cb']== "Yes" and  row['correct_xgb']=="No":
        nr_corr_cb_incorr_xgb += 1
        
    elif row['correct_cb']== "No" and  row['correct_xgb']=="Yes":
        nr_incorr_cb_corr_xgb += 1
        
    elif row['correct_cb']== "No" and  row['correct_xgb']=="No":
        nr_incorr_cb_incorr_xgb += 1

In [9]:
contingency_table_df=pd.DataFrame(data={"nr_correct_cb":["Yes/Yes","No/Yes"], 
                                        "nr_incorrect_cb":["Yes/No","No/No"]}, 
                                  index=["nr_correct_xgb","nr_incorrect_xgb"])
contingency_table_df

,nr_correct_cb,nr_incorrect_cb
nr_correct_xgb,Yes/Yes,Yes/No
nr_incorrect_xgb,No/Yes,No/No


In [10]:
contingency_table_df.iloc[0,0] = nr_corr_cb_corr_xgb
contingency_table_df.iloc[0,1] = nr_incorr_cb_corr_xgb
contingency_table_df.iloc[1,0] = nr_corr_cb_incorr_xgb
contingency_table_df.iloc[1,1] = nr_incorr_cb_incorr_xgb

contingency_table_df

,nr_correct_cb,nr_incorrect_cb
nr_correct_xgb,361819,14854
nr_incorrect_xgb,11529,111798


In [11]:
contingency_table = [[361819, 14854],
                     [11529, 111798]]

# Apply McNemar test
result = mcnemar(contingency_table, exact=False, correction=True)
print('Test statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
print()
# interpret the p-value
alpha = 0.05
if result.pvalue > alpha:
    print('Same proportions of errors for the used models (fail to reject H0). Therefore the performance difference between two models are not statistically significant.')

else:
    print("""Different proportions of errors for the used models (reject H0). 
Therefore the performance difference between two models are statistically significant.""")

Test statistic=418.791, p-value=0.000

Different proportions of errors for the used models (reject H0). 
Therefore the performance difference between two models are statistically significant.
